In [1]:
!pip install -q langchain-huggingface langchain-community ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
import os
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool
import requests

In [3]:
from google.colab import userdata
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')


In [4]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
)

In [5]:
model = ChatHuggingFace(llm=llm)

In [6]:
search_tool = DuckDuckGoSearchRun()

In [8]:
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """

  url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'
  response = requests.get(url)
  return str(response.json())

In [9]:
prompt = hub.pull("hwchase17/react")

In [10]:
agent = create_react_agent(
    llm=model,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [11]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

In [12]:
response = agent_executor.invoke({
    "input": "Find the capital of Madhya Pradesh, then find its current weather condition"
})
print("\n\nFinal Answer:")
print(response['output'])



> Entering new AgentExecutor chain...
Thought: To find the capital of Madhya Pradesh, I can use a search engine.
Action: duckduckgo_search
Action Input: "capital of Madhya Pradesh"After the independence of India, Madhya Pradesh was created in 1950 from the former British Central Provinces and Berar and the princely states of Makrai and Chhattisgarh, with Nagpur as … This list provides the capital cities of all the states and union territories in India, ordered alphabetically by state. The list also provides the population of the cities based on the most recent census conducted in India (2011). Bhopal is the capital city of Madhya Pradesh , known for its historical significance, lakes, and rich cultural heritage. It serves as the political and administrative hub, housing the Madhya Pradesh Legislative Assembly, Secretariat, and High Court Bench. Madhya Pradesh has 55 districts and its total area is around 3,08,252 square kilometers. Madhya Pradesh's capital and state headquarter is Bh